In [ ]:
from flask import Flask, render_template_string, request, jsonify
import pandas as pd
from fuzzywuzzy import fuzz, process
import webbrowser
from threading import Thread

# Initialize the Flask app
app = Flask(__name__)

# Load the data from Excel
file_path = r"C:\Users\Saleel01\Downloads\Leeds-20241110T220642Z-001\Leeds\Work Done\New folder\datasets\Datasetfinaldivya12_11.xlsx"
excel_data = pd.ExcelFile(file_path)

# Load each sheet into a DataFrame
disease_bodyfn_df = excel_data.parse('30-10-DISEASE VS BODYFN')
bodyfn_assistive_df = excel_data.parse('31-10 BODYFN vs assistive techn')
bodyfn_home_icf_df = excel_data.parse('31-10 BODYFN vs HOME ICF')

# Preprocessing helper function
def preprocess_text(text):
    return text.strip().lower()

# Normalize the dataset for fuzzy matching
disease_bodyfn_df['Disease'] = disease_bodyfn_df['Disease'].apply(preprocess_text)
bodyfn_assistive_df['Symptoms'] = bodyfn_assistive_df['Symptoms'].apply(preprocess_text)
bodyfn_home_icf_df['Symptoms'] = bodyfn_home_icf_df['Symptoms'].apply(preprocess_text)

# Helper function to find disease suggestions based on input
def get_disease_suggestions(query, limit=10):
    query = preprocess_text(query)
    disease_list = disease_bodyfn_df['Disease'].tolist()
    
    # Filter diseases that start with the query or contain the query
    filtered_diseases = [disease for disease in disease_list if query in disease]
    
    # Sort based on fuzzy matching score
    matches = process.extract(query, filtered_diseases, scorer=fuzz.partial_ratio, limit=limit)
    return [match[0] for match in matches if match[1] >= 50]

# Helper function to find the closest match using fuzzy matching
def find_closest_match(input_text, text_list, threshold=70):
    match = process.extractOne(input_text, text_list, scorer=fuzz.partial_ratio)
    return match[0] if match and match[1] >= threshold else None

# Function to retrieve details based on disease input
def get_disease_details(disease_name):
    disease_name = preprocess_text(disease_name)
    disease_list = disease_bodyfn_df['Disease'].tolist()
    matched_disease = find_closest_match(disease_name, disease_list)
    
    if matched_disease is None:
        return f"No information found for disease: {disease_name}", [], []

    disease_info = disease_bodyfn_df[disease_bodyfn_df['Disease'] == matched_disease]
    symptoms = disease_info['Symptoms'].values[0].split(', ') if not disease_info.empty else []
    assistive_mapping, home_function_mapping = [], []

    all_symptoms_assistive = bodyfn_assistive_df['Symptoms'].tolist()
    all_symptoms_home = bodyfn_home_icf_df['Symptoms'].tolist()

    for symptom in symptoms:
        symptom = preprocess_text(symptom)

        # Match symptoms to assistive technology
        matched_symptom_assistive = find_closest_match(symptom, all_symptoms_assistive, threshold=80)
        if matched_symptom_assistive:
            tech_info = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'] == matched_symptom_assistive]
            assistive_tech = tech_info['Assistive Technology'].values[0] if not tech_info.empty else "No assistive technology found"
        else:
            assistive_tech = "No match found"

        assistive_mapping.append((symptom, assistive_tech))

        # Match symptoms to home functions
        matched_symptom_home = find_closest_match(symptom, all_symptoms_home, threshold=80)
        if matched_symptom_home:
            home_info = bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'] == matched_symptom_home]
            home_functions = ", ".join(home_info['Home functions'].dropna().tolist()) or "No home functions found"
        else:
            home_functions = "No match found"

        home_function_mapping.append((symptom, home_functions))

    return matched_disease, assistive_mapping, home_function_mapping

# HTML template with real-time disease suggestion functionality and tabulated output
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HOMEAI-ENABLE</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            color: #333;
            padding: 20px;
        }
        .container {
            max-width: 800px;
            margin: 0 auto;
            background: #fff;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            padding: 20px;
        }
        h1 {
            text-align: center;
            color: #0056b3;
        }
        form {
            margin-top: 20px;
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        input[type="text"] {
            padding: 10px;
            width: 80%;
            margin-bottom: 10px;
            border-radius: 5px;
            border: 1px solid #ddd;
        }
        ul.suggestions {
            list-style-type: none;
            padding: 0;
            margin: -10px 0 10px;
            width: 80%;
            border: 1px solid #ddd;
            background: #fff;
            border-radius: 5px;
            max-height: 150px;
            overflow-y: auto;
        }
        ul.suggestions li {
            padding: 10px;
            cursor: pointer;
        }
        ul.suggestions li:hover {
            background: #f4f4f4;
        }
        button {
            padding: 10px 20px;
            background-color: #0056b3;
            color: #fff;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background-color: #003f88;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #0056b3;
            color: #fff;
        }
    </style>
    <script>
        async function fetchSuggestions(query) {
            if (!query) {
                document.getElementById('suggestions').innerHTML = '';
                return;
            }
            const response = await fetch(`/suggestions?query=${query}`);
            const suggestions = await response.json();
            const suggestionsList = suggestions.map(item => `<li onclick="selectSuggestion('${item}')">${item}</li>`).join('');
            document.getElementById('suggestions').innerHTML = `<ul class="suggestions">${suggestionsList}</ul>`;
        }
        function selectSuggestion(suggestion) {
            document.getElementById('disease_name').value = suggestion;
            document.getElementById('suggestions').innerHTML = '';
        }
    </script>
</head>
<body>
    <div class="container">
        <h1>HOMEAI-ENABLE</h1>
        <form method="POST">
            <input type="text" id="disease_name" name="disease_name" placeholder="Enter Disease Name" oninput="fetchSuggestions(this.value)" required>
            <div id="suggestions"></div>
            <button type="submit">Submit</button>
        </form>
        {% if message %}
            <p>{{ message }}</p>
        {% endif %}
        {% if disease %}
            <h2>Matched Disease: {{ disease }}</h2>
            <h3>Assistive Technologies</h3>
            <table>
                <thead>
                    <tr><th>Symptom</th><th>Assistive Technology</th></tr>
                </thead>
                <tbody>
                    {% for item in assistive_mapping %}
                        <tr><td>{{ item[0] }}</td><td>{{ item[1] }}</td></tr>
                    {% endfor %}
                </tbody>
            </table>
            <h3>Home Functions</h3>
            <table>
                <thead>
                    <tr><th>Symptom</th><th>Home Functions</th></tr>
                </thead>
                <tbody>
                    {% for item in home_function_mapping %}
                        <tr><td>{{ item[0] }}</td><td>{{ item[1] }}</td></tr>
                    {% endfor %}
                </tbody>
            </table>
        {% endif %}
    </div>
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        disease_name = request.form.get('disease_name')
        matched_disease, assistive_mapping, home_function_mapping = get_disease_details(disease_name)
        if isinstance(matched_disease, str) and "No information found" in matched_disease:
            return render_template_string(template, message=matched_disease)
        return render_template_string(
            template,
            disease=matched_disease.capitalize(),
            assistive_mapping=assistive_mapping,
            home_function_mapping=home_function_mapping,
            message=None
        )
    return render_template_string(template)

@app.route('/suggestions')
def suggestions():
    query = request.args.get('query', '')
    suggestions = get_disease_suggestions(query)
    return jsonify(suggestions)

# Function to run the Flask app and open the browser
def open_browser(port):
    webbrowser.open(f"http://localhost:{port}")

def run_app():
    port = 5011  # Port greater than 10
    thread = Thread(target=open_browser, args=(port,))
    thread.start()
    app.run(port=port)

# Run the app
run_app()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5011
Press CTRL+C to quit
